In [1]:
# Drive version:
# !pip uninstall TabularExperimentTrackerClient --y
# !pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
# !pip uninstall NeuralNetworksTrainingPackage --y
# !pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

In [2]:
%pip uninstall NeuralNetworksTrainingPackage --y
#%pip uninstall TabularExperimentTrackerClient --y
%pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
%pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

Found existing installation: NeuralNetworksTrainingPackage 1.0.0
Uninstalling NeuralNetworksTrainingPackage-1.0.0:
  Successfully uninstalled NeuralNetworksTrainingPackage-1.0.0
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/DanielWarfield1/TabularExperimentTrackerClient to /tmp/pip-req-build-1hect6vn
  Running command git clone --filter=blob:none --quiet https://github.com/DanielWarfield1/TabularExperimentTrackerClient /tmp/pip-req-build-1hect6vn
  Resolved https://github.com/DanielWarfield1/TabularExperimentTrackerClient to commit 780933411aa8c4e394478a26dec4a39447f8f012
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/Bartosz-G/NeuralNetworksTrainingPackage to /tmp/pip-req-build-3vn20w_9
  Running command git clone --filter=blob:none --qui

In [3]:
import numpy as np
import pandas as pd
import sklearn
import torch
import time

In [4]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../creds/'
creds_orch_file = "creds-orch.txt"
creds_openml_file = "creds-openml.txt"



with open(os.path.join(path, creds_orch_file), 'r') as file:
    lines = file.readlines()
    orchname = lines[0].strip()
    orchsecret = lines[1].strip()

with open (os.path.join(path, creds_openml_file), "r") as myfile:
    openMLAPIKey = myfile.read()

ex = ExperimentClient(verbose = True)


ex.define_orch_cred(orchname, orchsecret)
ex.define_opml_cred(openMLAPIKey)

# Colab version
# ex.define_opml_cred_drive('/My Drive/research/non-homogenous-data/creds/creds-openml.txt')
# ex.define_orch_cred_drive('bart', '/My Drive//research/non-homogenous-data/creds/creds-colab.txt')

# 1. Defining the experiment

In [5]:
experiment_name = 'experiment_LCN_1'



# LCN and LLN Parameters
depth = {'distribution': 'int_uniform', 'min':1, 'max':11}
seed = {'distribution': 'constant', 'value': 42}
drop_type = {'distribution': 'categorical', 'values':['node_dropconnect', 'none']}
p = {'distribution': 'float_uniform', 'min':0.25, 'max':0.75}
back_n = {'distribution': 'categorical', 'values':[0, 0, 0, 1]}
hidden_dim = {'distribution': 'constant', 'value': 1} # Assertion error coming from Net if not 1
anneal = {'distribution': 'categorical', 'values':['interpolation', 'none', 'approx']}
batch_size = {'distribution': 'categorical', 'values':[16,32,64,64,64,128,256]}
epochs = {'distribution': 'constant', 'value': 30}
lr = {'distribution': 'log_uniform', 'min':0.05, 'max':0.2} # yields mean = 0.1082, median 0.1
momentum = {'distribution': 'constant', 'value': 0.9}
no_cuda = {'distribution': 'constant', 'value': False}
lr_step_size = {'distribution': 'categorical', 'values':[10, 10, 15, 20]}
gamma = {'distribution': 'constant', 'value': 0.1}

In [6]:
#============================================================
# Regression: Locally Constant Networks
#============================================================
LCN_reg_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

LCN_reg_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

#============================================================
# Regression: Locally Linear Networks
#============================================================

LLN_reg_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }


LLN_reg_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

#============================================================
# Classification: Locally Constant Networks
#============================================================

LCN_cls_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

LCN_cls_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

#============================================================
# Classification: Locally Linear Networks
#============================================================

LLN_cls_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }


LLN_cls_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

In [7]:
model_groups = {
    'LCN_reg_SGD':{'model':'LCN_reg_SGD', 'hype':LCN_reg_SGD_space},
    'LCN_reg_AMSGrad':{'model':'LCN_reg_AMSGrad', 'hype':LCN_reg_AMSGrad_space},
    'LLN_reg_SGD':{'model':'LLN_reg_SGD', 'hype':LLN_reg_SGD_space},
    'LLN_reg_AMSGrad':{'model':'LLN_reg_AMSGrad', 'hype':LLN_reg_AMSGrad_space},
    'LCN_cls_SGD':{'model':'LCN_cls_SGD', 'hype':LCN_cls_SGD_space},
    'LCN_cls_AMSGrad':{'model':'LCN_cls_AMSGrad', 'hype':LCN_cls_AMSGrad_space},
    'LLN_cls_SGD':{'model':'LLN_cls_SGD', 'hype':LLN_cls_SGD_space},
    'LLN_cls_AMSGrad':{'model':'LLN_cls_AMSGrad', 'hype':LLN_cls_AMSGrad_space},
}

ex.def_model_groups(model_groups)

In [8]:
ex.def_data_groups_opml()
print(f'automatically defined data groups: {ex.data_groups.keys()}')

classification_models = [k for k in model_groups.keys() if '_cls' in k]
regression_models = [k for k in model_groups.keys() if '_reg' in k]


applications = {'opml_reg_purnum_group': regression_models,
                'opml_reg_numcat_group': regression_models,
                'opml_class_purnum_group': classification_models,
                'opml_class_numcat_group': classification_models}

ex.def_applications(applications)
ex.reg_experiment(experiment_name)

automatically defined data groups: dict_keys(['opml_reg_purnum_group', 'opml_class_purnum_group', 'opml_reg_numcat_group', 'opml_class_numcat_group'])
existing experiment found


'existing experiment found'

In [9]:
exp_info = ex.experiment_info()
successful_runs = exp_info['successful_runs']
required_runs = exp_info['required_runs']
print('total required runs: {}'.format(required_runs))

total required runs: 14160


In [10]:
# data_pre_processing.reset(True)

# 2. General Data pre-processing

In [11]:
from NeuralNetworksTrainingPackage.event_handler import dataPreProcessingEventEmitter
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import filterCardinality, quantileTransform, trunctuateData, oneHotEncodePredictors, oneHotEncodeTargets, toDataFrame, splitTrainValTest

n_sample = 20000
split = [0.5, 0.25, 0.25]
quantile_transform_distribution='uniform'


data_pre_processing = dataPreProcessingEventEmitter()

filter_cardinality = filterCardinality(transform = 'all')
trunctuate_data = trunctuateData(n = n_sample, transform = 'all')
one_hot_encode_predictors = oneHotEncodePredictors(transform = 'all')
one_hot_encode_targets = oneHotEncodeTargets(transform = 'all')
to_data_frame = toDataFrame(transform = 'all')
split_train_val_test = splitTrainValTest(split = split)
quantile_transform = quantileTransform(output_distribution = quantile_transform_distribution, transform = 'train')


# Transformations will be called in the order they're added to data_pre_processing
data_pre_processing.add_pre_processing_step('regression', filter_cardinality)
data_pre_processing.add_pre_processing_step('regression', trunctuate_data)
data_pre_processing.add_pre_processing_step('regression', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('regression', to_data_frame)
data_pre_processing.add_pre_processing_step('regression', split_train_val_test)
data_pre_processing.add_pre_processing_step('regression', quantile_transform)


data_pre_processing.add_pre_processing_step('classification', filter_cardinality)
data_pre_processing.add_pre_processing_step('classification', trunctuate_data)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_targets)
data_pre_processing.add_pre_processing_step('classification', to_data_frame)
data_pre_processing.add_pre_processing_step('classification', split_train_val_test)
data_pre_processing.add_pre_processing_step('classification', quantile_transform)

# 3. Model Specific Data pre-processing

In [12]:
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import CustomDataset, toPyTorchDatasets

to_pytorch_datasets = toPyTorchDatasets(wrapper = CustomDataset)

# Transformations will be called after general pre-processing steps, and in order they're added
data_pre_processing.add_pre_processing_step('LCN_reg_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_reg_AMSGrad', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_reg_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_reg_AMSGrad', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_cls_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_cls_AMSGrad', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_cls_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_cls_AMSGrad', to_pytorch_datasets)

# 4. Model Metrics

In [13]:
from metrics.LcnMetrics import LcnMetricsClassification, LcnMetricsRegression

lcn_metrics_regression = LcnMetricsRegression()
lcn_metrics_classification = LcnMetricsClassification()

lcn_metrics = {'regression': lcn_metrics_regression,
               'classification': lcn_metrics_classification}

metric_model_pairs = {
    'LCN_reg_SGD': lcn_metrics,
    'LCN_reg_AMSGrad': lcn_metrics,
    'LLN_reg_SGD': lcn_metrics,
    'LLN_reg_AMSGrad': lcn_metrics,
    'LCN_cls_SGD': lcn_metrics,
    'LCN_cls_AMSGrad': lcn_metrics,
    'LLN_cls_SGD': lcn_metrics,
    'LLN_cls_AMSGrad': lcn_metrics,
}

# 5. Model Training routine

In [14]:
from models.LcnNetwork import init_LcnNetwork
from training.LcnTrain import LcnTrainingRoutine

lcn_model_and_training = {'model_init':init_LcnNetwork, 'training_routine': LcnTrainingRoutine}

model_training_pairs = {
    'LCN_reg_SGD': lcn_model_and_training,
    'LCN_reg_AMSGrad': lcn_model_and_training,
    'LLN_reg_SGD': lcn_model_and_training,
    'LLN_reg_AMSGrad': lcn_model_and_training,
    'LCN_cls_SGD': lcn_model_and_training,
    'LCN_cls_AMSGrad': lcn_model_and_training,
    'LLN_cls_SGD': lcn_model_and_training,
    'LLN_cls_AMSGrad': lcn_model_and_training,
}

# 6. Main Experiment loop

In [ ]:


torch_models = ('LCN_reg_SGD', 'LCN_reg_AMSGrad', 'LLN_reg_SGD', 'LLN_reg_AMSGrad', 'LCN_cls_SGD','LCN_cls_AMSGrad', 'LLN_cls_SGD','LLN_cls_AMSGrad')

sklearn_models = ('no_sklearn_models_in_this_training')

for i in range(14160):
    print(f'==== Begin run:{i} ====')
    run_info = ex.begin_run_sticky()

    hyperparameters = run_info['hyp']
    model_name = run_info['model']

    task = hyperparameters['task']
    seed = hyperparameters['seed']



    torch.manual_seed(seed)
    np.random.seed(seed)


    print('---- Loading datasets ----')
    X, y, categorical_indicator, attribute_names = ex.opml_load_task(run_info['mtpair_task'])

    # Pre-processing
    data_pre_processing.set_seed_for_all(seed)
    data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
    data_pre_processing.apply(task)
    data_pre_processing.apply(model_name)
    train_data, val_data, test_data = data_pre_processing.get_train_val_test()


    # Getting appropriate metrics
    metrics_calculator = metric_model_pairs[model_name][task]


    match model_name:
        case _ if model_name in sklearn_models:
            pass

        case _ if model_name in torch_models:
            # hyperparameters will be updated with {'input_dim': num_columns_X, 'output_dim':num_columns_Y}
            hyperparameters.update(train_data.get_dims())

            train_batch_size = hyperparameters['batch_size']
            train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=train_batch_size,shuffle= True)
            val_dataloader = torch.utils.data.DataLoader(val_data,batch_size=len(val_data),shuffle= True)
            test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=len(test_data),shuffle= True)


            init_model = model_training_pairs[model_name]['model_init']
            TrainingRoutine = model_training_pairs[model_name]['training_routine']


            model = init_model(**hyperparameters)
            training_routine = TrainingRoutine(**hyperparameters)
            
            training_routine.set_optimizer_scheduler(model)

            start_epoch = 1  # start from epoch 1 or last checkpoint epoch
            total_epochs = hyperparameters['epochs']
            start_time = time.time()

            for epoch in range(start_epoch, total_epochs + start_epoch):
                print(f"----{epoch}th training epoch ----")
                epoch_metrics = {}

                training_routine.scheduler_step(epoch)
                train_loss = training_routine.train(model, train_dataloader)


                if train_loss is None:
                    print('---Stopping training due to loss being nan!---')
                    epoch_metrics = {'train_loss': train_loss, 'epoch': epoch}
                    ex.update_run(epoch_metrics)
                    break

                if epoch == total_epochs:
                    continue

                epoch_metrics.update(train_loss)
                epoch_metrics.update({'epoch': epoch})
                ex.update_run(epoch_metrics)
                print(epoch_metrics)

            else:
                final_metrics = {}
                training_time = time.time()-start_time

                train_metrics = metrics_calculator.get_metrics(model, train_dataloader, hyperparameters, 'train')
                val_all_metrics = metrics_calculator.get_all(model, val_dataloader, hyperparameters, 'val')
                test_all_metrics = metrics_calculator.get_all(model, test_dataloader, hyperparameters, 'test')

                final_metrics.update(train_loss)
                final_metrics.update(train_metrics)
                final_metrics.update(val_all_metrics)
                final_metrics.update(test_all_metrics)
                final_metrics.update({'epoch': epoch})
                final_metrics.update({'epoch_time': training_time})

                ex.update_run(final_metrics)
                print(final_metrics)

    ex.end_run()

==== Begin run:0 ====
{'_id': '6500872c2b022a2902237f3b', 'metrics_per_epoch': [], 'experiment_id': '64f0f2556e02727fe9a6ff59', 'experiment_name': 'experiment_LCN_1', 'mtpair_index': 111, 'mtpair_model': 'LLN_reg_AMSGrad', 'mtpair_task': '335-361103', 'is_completed': False, 'user_id': '64d3a7457658d6ec6db139d0', 'user_name': 'bart', 'hyp': {'depth': 5, 'seed': 42, 'drop_type': 'node_dropconnect', 'p': 0.483507611463855, 'ensemble_n': 1, 'shrinkage': 1, 'back_n': 0, 'net_type': 'locally_linear', 'hidden_dim': 1, 'anneal': 'approx', 'optimizer': 'AMSGrad', 'batch_size': 256, 'epochs': 30, 'lr': 0.1261993572352542, 'momentum': 0.9, 'no_cuda': False, 'lr_step_size': 15, 'gamma': 0.1, 'task': 'regression'}, 'model': 'LLN_reg_AMSGrad', 'task': '335-361103'}
500872c2b022a2902237f3
---- Loading datasets ----
downloading task 335-361103
task different than previous task, downloading...
----1th training epoch ----
run updated
{'train_loss': 0.9214727756023406, 'epoch': 1}
----2th training epoch 

## Checking whether the code works as intended

In [40]:
def get_metrics(model, test_loader, hyperparameters, test_set_name=None):
        if test_set_name:
            assert isinstance(test_set_name, str), "test_set_name must be a string, such as train, val, test"
            
        no_cuda = hyperparameters['no_cuda']    
        use_cuda = not no_cuda and torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")

        with torch.no_grad():
            model.eval()
            test_loss = 0
            dataset_len = 0
            standard_errors_list = []
            target_list = []

            for data, target in test_loader:
                dataset_len += len(target)
                data, target = data.to(device), target.to(device)

                target_list.append(target)

                ###############
                data.requires_grad = True
                if model.net_type == 'locally_constant':
                    output, relu_masks = model(data, p=0, training=False)
                elif model.net_type == 'locally_linear':
                    output, relu_masks = model.normal_forward(data, p=0, training=False)
                ###############

                standard_errors = ((output - target) ** 2)
                # print(standard_errors.shape)
                # print(target.shape)
                standard_errors_list.append(standard_errors)
                test_loss += ((output - target) ** 2).mean().item() * len(target)

            test_loss /= dataset_len
            
            standard_errors_tensor = torch.cat(standard_errors_list, dim=0)
            all_targets = torch.cat(target_list, dim=0)

            
            standard_errors_tensor = torch.squeeze(standard_errors_tensor)
            all_targets = torch.squeeze(all_targets)
            
            print(standard_errors_tensor.shape)
            print(all_targets.shape)


            ss_res = torch.sum(standard_errors_tensor)
            target_mean = torch.mean(all_targets)
            ss_tot = torch.sum((all_targets - target_mean) ** 2)
            r2_score = 1 - (ss_res / ss_tot)
            
            # print(ss_res)
            # print(target_mean)
            # print(ss_tot)
            # print(r2_score)

            RMSE = float(np.sqrt(test_loss))

            quantiles = torch.tensor(self.quantiles, device=device)
            quantile_values = torch.quantile(standard_errors_tensor, quantiles)

            # standard_errors_pd = pd.DataFrame(standard_errors_tensor.cpu().numpy())
            # quantile_dict = standard_errors_pd.quantile(self.quantiles).to_dict()

            quantile_dict = {q: float(v.item()) for q, v in zip(self.quantiles, quantile_values)}

            metrics = {
                'RMSE': RMSE,
                'r2_score': r2_score.cpu().item(),
                'se_quant': quantile_dict}

            if test_set_name:
                assert isinstance(test_set_name, str), "test_set_name must be a string, such as 'train', 'val', 'test'"
                metrics_name = f'{test_set_name}_metrics'
                final_metrics = {metrics_name: metrics}
            else:
                final_metrics = {'metrics': metrics}


            return final_metrics


In [41]:
metrics = get_metrics(model, train_dataloader, hyperparameters, 'train')

torch.Size([4320])
torch.Size([4320])


NameError: name 'self' is not defined

In [66]:
model = init_model(**hyperparameters)

net_type approx_locally_constant is not supported


NameError: name 'exit' is not defined

In [16]:
X, y, categorical_indicator, attribute_names = ex.opml_load_task('335-361097')

downloading task 335-361097
using values from previous task load, skipped download


In [17]:
from copy import deepcopy
#X_c, y_c, categorical_indicator_c, attribute_names_c = deepcopy(X), deepcopy(y), deepcopy(categorical_indicator), deepcopy(attribute_names)
X, y, categorical_indicator, attribute_names = deepcopy(X_c), deepcopy(y_c), deepcopy(categorical_indicator_c), deepcopy(attribute_names_c)

In [18]:
X_c

,X3,X4,X6,X10,X12,X13,X14,X15,X16,X17,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,3,9,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,4,3,11,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,3,9,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,5,3,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,3,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,2,3,3,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4205,3,3,7,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4206,0,3,6,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4207,5,3,11,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
y_c

0       130.81
1        88.53
2        76.26
3        80.62
4        78.02
         ...  
4204    107.39
4205    108.77
4206    109.22
4207     87.48
4208    110.85
Name: y, Length: 4209, dtype: float64

In [20]:
categorical_indicator

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [56]:
data_pre_processing.set_seed_for_all(seed)
data_pre_processing.reset()
data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
data_pre_processing.apply(task)
data_pre_processing.apply(model_name)

In [74]:
data_pre_processing.reset()

In [57]:
print(data_pre_processing.val)

In [58]:
train_data, val_data, test_data = data_pre_processing.get_train_val_test()

In [59]:
train_data

In [60]:
X_out, y_out, categorical_indicator_out, attribute_names_out = data_pre_processing.get('train')

ValueError: too many values to unpack (expected 4)

In [30]:
X_out

,bedrooms,bathrooms,sqft_living,sqft_lot,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,date_month,date_day,waterfront_0,waterfront_1,date_year_0,date_year_1
0,0.366867,0.095596,0.289790,0.299354,0.060060,0.021021,0.823824,0.167668,0.0,0.419110,0.152653,0.024525,0.672673,0.652152,0.576577,1.0,0.0,1.0,0.0
1,0.366867,0.436937,0.557057,0.212212,0.312813,0.133634,0.918418,0.093093,0.0,0.453453,0.086587,0.365365,0.204204,0.556056,0.446947,1.0,0.0,1.0,0.0
2,0.072573,0.095596,0.619620,0.818804,0.312813,0.451952,0.803303,0.335836,0.0,0.150150,0.334835,0.475976,0.473056,0.145646,0.045045,1.0,0.0,0.0,1.0
3,0.366867,0.928929,0.227227,0.016607,0.663163,0.385385,0.000000,0.921421,0.0,0.706599,0.192192,0.130130,0.029824,0.453954,0.576577,1.0,0.0,1.0,0.0
4,0.754755,0.898899,0.472472,0.713093,0.663163,0.636637,0.000000,0.696196,0.0,0.318102,0.707708,0.717217,0.622752,0.073574,0.745746,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.366867,0.436937,0.361361,0.167024,0.312813,0.073073,0.816817,0.158158,0.0,0.691311,0.209710,0.386386,0.130130,0.348348,0.415916,1.0,0.0,0.0,1.0
9996,0.366867,0.702202,0.507508,0.035868,0.663163,0.327327,0.777778,0.861862,0.0,0.683000,0.064565,0.358859,0.337838,0.556056,0.078579,1.0,0.0,1.0,0.0
9997,0.990490,0.854354,0.741241,0.662169,0.312813,0.341341,0.949950,0.374374,0.0,0.974665,0.370370,0.488989,0.798799,0.453954,0.609109,1.0,0.0,1.0,0.0
9998,0.366867,0.323323,0.177678,0.816407,0.312813,0.313313,0.000000,0.525526,0.0,0.192898,0.571572,0.394394,0.820877,0.073574,0.110110,1.0,0.0,0.0,1.0


In [33]:
y_out

,price
0,12.506181
1,13.270785
2,12.524530
3,13.085701
4,12.971543
...,...
9995,13.377007
9996,12.994510
9997,12.923915
9998,12.611541


In [34]:
categorical_indicator_out

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True]

In [26]:
data_pre_processing.events

{'regression': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.filterCardinality at 0x7fba20983c40>,
 'classification': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.filterCardinality at 0x7fba20983c40>,
 'LCN_reg_SGD': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.toPyTorchDatasets at 0x7fba20163430>],
 'LCN_reg_AMSGrad': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.toPyTorchDatasets at 0x7fba20163430>],
 'LLN_reg_SGD': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.toPyTorchDatasets at 0x7fba20163430>],
 'LLN_reg_AMSGrad': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.toPyTorchDatasets at 0x7fba20163430>],
 'LCN_cls_SGD': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.toPyTorchDatasets at 0x7fba20163430>],
 'LCN_cls_AMSGrad': [<NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing.toPyTorchDatasets at 0x7fba20163430>],
 'LL

Net(
  (layer): ModuleList(
    (0): Linear(in_features=19, out_features=1, bias=True)
    (1): Linear(in_features=20, out_features=1, bias=True)
    (2): Linear(in_features=21, out_features=1, bias=True)
    (3): Linear(in_features=22, out_features=1, bias=True)
    (4): Linear(in_features=23, out_features=1, bias=True)
    (5): Linear(in_features=24, out_features=1, bias=True)
  )
  (output_fc): Linear(in_features=25, out_features=1, bias=True)
)